This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify plane quintic curves (of genus 6) which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 5 minutes for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $\mathbf{P}^2$.

In [2]:
F = GF(2)
P.<x0,x1,x2> = F[]

In [3]:
S = [vector(t) for t in ProjectiveSpace(F, 2).rational_points()]
for v in S:
    v.set_immutable()
len(S)

7

Use an orbit lookup tree to find $\mathrm{GL}(3,\mathbb{F}_2)$-orbit representatives for 6-tuples of $\mathbb{F}_2$-points in $\mathbf{P}^2$.

In [4]:
def apply_group_elem(g, x):
    x1 = g*x
    x1.set_immutable()
    return x1

In [5]:
def stabilizer(x):
    return GL(3, 2).subgroup([g.matrix().transpose() for g in vec_stab(Matrix(x)).gens()])

In [6]:
def optimized_rep(g):
    return g.matrix()

In [7]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}

In [8]:
G = GL(3, F)
tree = build_orbit_tree(G, S, 6, methods, verbose=False)

For each representative tuple, solve a linear system to find all quintics passing through exactly these $\mathbb{F}_2$-rational points.

In [9]:
monos5 = [prod(x) for x in itertools.combinations_with_replacement(P.gens(), 5)]
len(monos5)

21

In [10]:
coords5 = {x: vector(mu(*x) for mu in monos5) for x in S}

In [11]:
def vec_to_gen(v):
    return sum(v[i]*monos5[i] for i in range(len(monos5)))

In [12]:
curves = defaultdict(list)
perp = Matrix([coords5[x] for x in S])
for s in range(4, 7):
    for vecs in green_nodes(tree, s):
        target = vector(F, (0 if x in vecs else 1 for x in S))
        for w in solve_right_iterator(perp, target):
            curves[(s,)].append(vec_to_gen(w))

In [13]:
[(s, len(curves[s])) for s in curves]

[((4,), 32768), ((5,), 16384), ((6,), 16384)]

Compute point counts over $\mathbb{F}_{q^i}$ for $i=2,3,4$, retaining cases that fit our targets.

In [14]:
def count_by_ideal(gen, n):
    J = P.ideal([gen] + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [15]:
for n in range(2, 5):
    tmp = set(t[:n] for t in targets6)
    tmp2 = list(curves.keys())
    for s in tmp2:
        for gen in curves[s]:
            i = count_by_ideal(gen, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gen)
        del curves[s]
    print([(s, len(curves[s])) for s in curves])

[((4, 14), 360), ((4, 16), 24), ((5, 11), 2880), ((5, 13), 920), ((5, 15), 192), ((6, 10), 5064), ((6, 14), 920)]
[((4, 14, 16), 63), ((4, 16, 16), 3), ((5, 11, 11), 768), ((5, 13, 14), 200), ((5, 15, 5), 16), ((6, 10, 9), 888), ((6, 14, 12), 56), ((6, 14, 6), 36)]
[((4, 14, 16, 18), 24), ((5, 11, 11, 31), 196), ((5, 11, 11, 39), 4), ((5, 13, 14, 25), 32), ((6, 14, 12, 26), 4)]


Close out this case.

In [16]:
proj = magma.ProjectiveSpace(P)
closeout(curves, X=proj)

Number of curves found: 8
Number of isomorphism classes found: 1
No covers found in this case!
Total time: 58 seconds
